# Example notebook for supervised training of LTC cells for time-series prediction tasks
#### *(using the Pytorch implementation)*

### Import requirements

In [3]:
from ray.rllib.agents.ppo import PPOTrainer

In [4]:
# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "Taxi-v3",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 2,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    }
}

# Create our RLlib Trainer.
trainer = PPOTrainer(config=config)

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for _ in range(3):
    print(trainer.train())

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
trainer.evaluate()


2022-03-07 17:51:41,672	WARNING trainer.py:2279 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
(pid=73403) 
(pid=73404) 
(RolloutWorker pid=73403) 2022-03-07 17:51:51,457	ERROR worker.py:432 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=73403, ip=127.0.0.1)
(RolloutWorker pid=73403)   File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 460, in __init__
(RolloutWorker pid=73403)     self.env = env_creator(copy.deepcopy(self.env_context))
(RolloutWorker pid=73403)   File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/env/utils.py", line 52, in gym_env_creator
(RolloutWorker pid=734

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=73404, ip=127.0.0.1)
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 460, in __init__
    self.env = env_creator(copy.deepcopy(self.env_context))
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/ray/rllib/env/utils.py", line 52, in gym_env_creator
    return gym.make(env_descriptor, **env_context)
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/gym/envs/registration.py", line 676, in make
    return registry.make(id, **kwargs)
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/gym/envs/registration.py", line 520, in make
    return spec.make(**kwargs)
  File "/Users/renard/miniconda3/envs/torchLTC/lib/python3.8/site-packages/gym/envs/registration.py", line 139, in make
    cls = load(self.entry_point)
  File "/Users/